In [1]:
import os
import tqdm
import glob
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import gc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
%matplotlib inline
plt.style.use('bmh')
import seaborn as sns

In [2]:
# parent directory
pdir = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction'

from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

# load meta data

In [3]:
df_tdcs_meta = pd.read_csv(os.path.join(pdir, 'tdcsfog_metadata.csv'))
#shape 833,5
#df_tdcs_meta.shape

#df_tdcs_meta.isna().sum(), no missing, 833#
#pd.unique(df[['Col1', 'Col2']].values.ravel('K')) or np.unique(df[['Col1', 'Col2']].values)-slower or df.nunique()
#df_tdcs_meta.groupby(['Subject']).mean().plot.hist()
#df_tdcs_meta.groupby(['Id']).nunique()
#df_tdcs_meta.groupby(['Subject']).nunique().plot.hist()
df_defog_meta = pd.read_csv(os.path.join(pdir, 'defog_metadata.csv'))
#df_defog_meta.shape
#(137, 4)
#print(type(df_defog_meta))
#df_defog_meta.groupby(['Subject']).mean().shape
#df_tdcs_meta.head()
df_defog_meta.head()

,Id,Subject,Visit,Medication
0,02ab235146,e1f62e,2,on
1,02ea782681,ae2d35,2,on
2,06414383cf,8c1f5e,2,off
3,092b4c1819,2874c5,1,off
4,0a900ed8a2,0e3d49,2,on


In [4]:
#subject.nunique=62 num =833 s
#temp0=df_tdcs_meta['Subject']
#.nunique()
#temp0=temp0.tolist()
#lent=len(temp0)
#temp0=set(temp0)
#temp0
#subject.nunique=45 numb =137
#tempd=df_defog_meta['Subject']
#tempd

In [5]:
#df_tdcs_meta.groupby(['Id']).nunique()
#un = df_tdcs_meta.groupby(['Subject']).nunique()



There are no NAN values (missing) as all features/columns have 833 count.

In [6]:
#print(df_tdcs_meta["Visit"].describe())
# plt.figure(figsize=(5,4))
# sns.displot(df_tdcs_meta['Visit'],  kde=True,
#     stat="density", color='g', bins=100, kde_kws= dict(cut=3), alpha=0.4);
# df_num = df_tdcs_meta.select_dtypes(include = ['float64', 'int64'])
# df_num.head()
# df_num.hist(figsize=(16,20), bins=50,xlabelsize=8, ylabelsize=8);

In [7]:
# print(df_tdcs_meta["Medication"].describe())
# plt.figure(figsize=(9,8))
# sns.displot(df_tdcs_meta['Medication'],  kde=True,
#     stat="density", color='g', bins=100, kde_kws= dict(cut=3), alpha=0.4);
# plt.figure(figsize=(3,2))
# sns.displot(df_tdcs_meta['Visit'],  kde=True,
#     stat="density", color='g', bins=100, kde_kws= dict(cut=3), alpha=0.4);

In [8]:
# print(df_tdcs_meta["Visit"].describe())
# plt.figure(figsize=(9,8))
# sns.displot(df_tdcs_meta['Visit'],  kde=True,
#     stat="density", color='g', bins=100, kde_kws= dict(cut=3), alpha=0.4);

# plt.figure(figsize=(9,8))
# sns.displot(df_tdcs_meta['Visit'],  kde=True,
#     stat="density", color='g', bins=100, kde_kws= dict(cut=3), alpha=0.4);

In [9]:
df_subjects = pd.read_csv(os.path.join(pdir, 'subjects.csv'))
df_subjects.head()
# df_subjects.shape
# df_subjects.groupby(['Subject']).mean().info()
# #df_subjects.info()
#un1 = df_subjects['Subject'].nunique() #136


,Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,00f674,2.0,63,M,27.0,43.0,49.0,24
1,00f674,1.0,63,M,27.0,31.0,30.0,26
2,02bc69,NaN,69,M,4.0,21.0,NaN,22
3,040587,2.0,75,M,26.0,52.0,69.0,21
4,040587,1.0,75,M,26.0,47.0,75.0,24


In [10]:
#df_subjects.isna().sum()
#df_subjects.info()
#df_subjects.groupby(['Subject']).nunique()
#temp=df_subjects['Subject']
#temp=(temp.tolist())
#x=len(temp)
#print(lent-x)
#temp=set(temp)
#temp-temp0

In [11]:
#df_subjects.describe()

# Load tdcsfog data

In [12]:
# list of all tdcsfog csv file path
tdcs_file_path = glob.glob(os.path.join(pdir, 'train', 'tdcsfog', '*.csv'), recursive=True)

# In this notebook, we limit the number of files to be read in order to reduce the time required for model training.
tdcs_file_path = tdcs_file_path[::50]

print(f'the number of files to be read: {len(tdcs_file_path)}')

the number of files to be read: 17


In [13]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_tdcs = pd.DataFrame()
#print(df_tdcs)

# load tdcsfog time series in combination with metadata.
for fp in tqdm.tqdm(tdcs_file_path):    
    
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    #print(tmp)
    #break
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    
    # add metadata.
    tmp['Medication'] = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] = df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    tmp['UPDRSIII_Off'] = df_subjects.loc[df_subjects['Subject'] == subject, 'UPDRSIII_Off'].iloc[0]
    tmp['UPDRSIII_On'] = df_subjects.loc[df_subjects['Subject'] == subject, 'UPDRSIII_On'].iloc[0]
    # concat the data
    df_tdcs = pd.concat([df_tdcs, tmp]).reset_index(drop=True)
#     print(df_tdcs)
#     break
#df_tdcs.isna().sum() UPDRSIII_Off=22080
#df_tdcs[['Time', 'AccV']]

100%|██████████| 17/17 [00:00<00:00, 22.69it/s]


In [14]:
# check the contents of the df_tdcs
# df_tdcs.head()
# df_tdcs.info()
#df_tdcs.shape
#df_tdcs.isna().sum(), UPDRSIII_Off=1435832, UPDRSIII_on=186621
#at 50, 17 files =shape(105537, 14), at 25 (230908, 14) at 0 7062672,14)
#df_tdcs.columns
#df_tdcs.groupby('Medication')[['UPDRSIII_Off', 'UPDRSIII_On']].mean().plot()
#x=df_tdcs[['Medication','UPDRSIII_Off']]
#x.describe()
#df_tdcs[['Medication','UPDRSIII_On']].describe
#df_tdcs['UPDRSIII_Off'].info()
#df_tdcs['UPDRSIII_On'].info()
#df_tdcs['UPDRSIII_On'].hist()

#plot.scatter('Medication', 'UPDRSIII_Off')

# Load defog data

In [15]:
# list of all tdcsfog csv file path
defog_file_path = glob.glob(os.path.join(pdir, 'train', 'defog', '*.csv'), recursive=True)

# In this notebook, we limit the number of files to be read in order to reduce the time required for model training.
defog_file_path = defog_file_path[::50]

print(f'the number of files to be read: {len(defog_file_path)}')

the number of files to be read: 2


In [16]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_defog = pd.DataFrame()

for fp in tqdm.tqdm(defog_file_path):
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] =df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    tmp['UPDRSIII_Off'] = df_subjects.loc[df_subjects['Subject'] == subject, 'UPDRSIII_Off'].iloc[0]
    tmp['UPDRSIII_On'] = df_subjects.loc[df_subjects['Subject'] == subject, 'UPDRSIII_On'].iloc[0]
    # extract data from the time period where Valid and Task are both True.
    tmp = tmp[(tmp['Valid'] == True) & (tmp['Task']==True)]
    tmp = tmp.drop(['Valid', 'Task'], axis=1)
    
    # concat the data
    df_defog = pd.concat([df_defog, tmp]).reset_index(drop=True)
    del tmp
    gc.collect()

100%|██████████| 2/2 [00:00<00:00,  2.72it/s]


In [17]:
# check the contents of the df_defog
#df_defog.shape
# at 50 (55399, 14) at 0 step (4090530, 14)
#df_defog[['Time','AccV']].isna().sum() 0

In [18]:
#df_defog.isna().sum() no na

# Prepare train data

In [19]:
# concat tdcs and defog data.
df_train = pd.concat([df_tdcs, df_defog]).reset_index(drop=True)
#df_train.head()
#df_train.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)


In [20]:
df_train['Medication'] = np.where(df_train['Medication']=='on', 1, 0)
df_train['Sex'] = np.where(df_train['Sex']=='M', 1, 0)
#df_train.head()

In [21]:
#df_train.columns


In [22]:
# split data into features and target.
df_train['Medication'] = np.where(df_train['Medication']=='on', 1, 0)
df_train['Sex'] = np.where(df_train['Sex']=='M', 1, 0)
y = df_train[['StartHesitation', 'Turn', 'Walking']]                       # target
X = df_train.drop(['StartHesitation', 'Turn', 'Walking'], axis=1)  # feature
X.interpolate()

del df_train
gc.collect()


126

In [23]:
# check the contents of the target
#y.head()
X.shape

(160936, 11)

In [24]:
# check the contents of the feature


# Train RandomForestClassifier

Change to xBoost

In [25]:
from sklearn.preprocessing import MaxAbsScaler
#scaler_gnb = MaxAbsScaler()
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline


scaler = MinMaxScaler()
sdss = scaler.fit_transform(X)


In [26]:
#rf= RandomForestClassifier()

# from sklearn.impute import SimpleImputer
# my_imputer = SimpleImputer()
# data_with_imputed_values = my_imputer.fit_transform(D)

# mlp= MLPClassifier(max_iter=2000, hidden_layer_sizes=(100,1000), alpha=1e-4,activation ='relu' ,  solver='adam', random_state=3)
# #X.isnull().sum()
# #y.isnull().sum()
# mlp.fit(X,y)
# print("accuracy", mlp.score(X,y))
# del mlp
# gc.collect()

In [27]:
xgb = XGBClassifier(n_estimators=100)
training_start = time.perf_counter()
xgb.fit(X, y)
training_end = time.perf_counter()


# Prepare test data

## Process the test data in the same way as the training data

In [28]:
# list of all tdcsfog csv file path
tdcs_test_file_path = glob.glob(os.path.join(pdir, 'test', 'tdcsfog', '*.csv'), recursive=True)
print(f'the number of files to be read: {len(tdcs_test_file_path)}')

the number of files to be read: 1


In [29]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_tdcs_test = pd.DataFrame()

for fp in tqdm.tqdm(tdcs_test_file_path):
    
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] = df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    
    # add Id data to submit.
    tmp['Id'] = file_id + '_' + tmp['Time'].astype(str)
    
    # concat the data
    df_tdcs_test = pd.concat([df_tdcs_test, tmp]).reset_index(drop=True)
    del tmp
    gc.collect()

100%|██████████| 1/1 [00:00<00:00,  6.52it/s]


In [30]:
# check the contents of the df_tdcs_test
#df_tdcs_test.head()

In [31]:
# list of all tdcsfog csv file path
defog_test_file_path = glob.glob(os.path.join(pdir, 'test', 'defog', '*.csv'), recursive=True)
print(f'the number of files to be read: {len(defog_test_file_path)}')

the number of files to be read: 1


In [32]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_defog_test = pd.DataFrame()

for fp in tqdm.tqdm(defog_test_file_path):
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] =df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    tmp['UPDRSIII_Off'] = df_subjects.loc[df_subjects['Subject'] == subject, 'UPDRSIII_Off'].iloc[0]
    tmp['UPDRSIII_On'] = df_subjects.loc[df_subjects['Subject'] == subject, 'UPDRSIII_On'].iloc[0]
    # add Id data to submit.
    tmp['Id'] = file_id + '_' + tmp['Time'].astype(str)
    
    # concat the data
    df_defog_test = pd.concat([df_defog_test, tmp]).reset_index(drop=True)
    

100%|██████████| 1/1 [00:00<00:00,  1.02it/s]


In [33]:
# check the contents of the df_defog_test
df_defog_test.head()

,Time,AccV,AccML,AccAP,Medication,Age,Sex,YearsSinceDx,NFOGQ,UPDRSIII_Off,UPDRSIII_On,Id
0,0,-0.914652,-0.300851,0.298156,on,79,F,8.0,28,55.0,52.0,02ab235146_0
1,1,-0.914367,-0.301572,0.298325,on,79,F,8.0,28,55.0,52.0,02ab235146_1
2,2,-0.779540,-0.343197,0.275690,on,79,F,8.0,28,55.0,52.0,02ab235146_2
3,3,-0.993162,-0.270281,0.315775,on,79,F,8.0,28,55.0,52.0,02ab235146_3
4,4,-0.948767,-0.307849,0.297191,on,79,F,8.0,28,55.0,52.0,02ab235146_4


In [34]:
# concat tdcs and defog data.
df_test = pd.concat([df_tdcs_test, df_defog_test]).reset_index(drop=True)

# encode string columns into 0/1 format
df_test['Medication'] = np.where(df_test['Medication']=='on', 1, 0)
df_test['Sex'] = np.where(df_test['Sex']=='M', 1, 0)
display(df_test)

,Time,AccV,AccML,AccAP,Medication,Age,Sex,YearsSinceDx,NFOGQ,Id,UPDRSIII_Off,UPDRSIII_On
0,0,-9.533939,0.566322,-1.413525,1,68,0,9.0,15,003f117e14_0,NaN,NaN
1,1,-9.536140,0.564137,-1.440621,1,68,0,9.0,15,003f117e14_1,NaN,NaN
2,2,-9.529345,0.561765,-1.429332,1,68,0,9.0,15,003f117e14_2,NaN,NaN
3,3,-9.531239,0.564227,-1.415490,1,68,0,9.0,15,003f117e14_3,NaN,NaN
4,4,-9.540825,0.561854,-1.429471,1,68,0,9.0,15,003f117e14_4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
286365,281683,-0.899299,-0.232668,0.389147,1,79,0,8.0,28,02ab235146_281683,55.0,52.0
286366,281684,-0.901973,-0.233051,0.390114,1,79,0,8.0,28,02ab235146_281684,55.0,52.0
286367,281685,-0.901690,-0.231888,0.391896,1,79,0,8.0,28,02ab235146_281685,55.0,52.0
286368,281686,-0.899576,-0.232429,0.391106,1,79,0,8.0,28,02ab235146_281686,55.0,52.0


In [35]:
# split data into submission Id and feature.
Id = df_test['Id']                             # Id for submission data
                       
X_test = df_test.drop(['Id'], axis=1)  # feature of test data

#model = LogisticRegression()

# Predict and submit

In [36]:
# calculate prediction using trained RandomForestClassifier model.
#pred = rf.predict(X_test)

# X_test.interpolate()



prediction_start = time.perf_counter()
pred2 = xgb.predict(X_test)
prediction_end = time.perf_counter()
#acc_xgb = (pred2 == X_test).sum().astype(float) / len(pred2)*100
print(X_test.shape, pred2.shape)
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start
#print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))
print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))


(286370, 11) (286370, 3)
Time consumed for training: 17.444
Time consumed for prediction: 0.42601 seconds


In [37]:
#print(rf.score(X_test, pred))
#print(mlp.score(X_test, pred2))
del X_test
gc.collect()

63

In [38]:
# Prepare submit data
submit = pd.DataFrame(Id, columns=['Id'])
submit['StartHesitation'] = pred2[:, 0]
submit['Turn'] = pred2[:, 1]
submit['Walking'] = pred2[:, 2]

In [39]:
display(submit)

,Id,StartHesitation,Turn,Walking
0,003f117e14_0,0.0,0.0,0.0
1,003f117e14_1,0.0,0.0,0.0
2,003f117e14_2,0.0,0.0,0.0
3,003f117e14_3,0.0,0.0,0.0
4,003f117e14_4,0.0,0.0,0.0
...,...,...,...,...
286365,02ab235146_281683,0.0,0.0,0.0
286366,02ab235146_281684,0.0,0.0,0.0
286367,02ab235146_281685,0.0,0.0,0.0
286368,02ab235146_281686,0.0,0.0,0.0


In [40]:
# Save the created submission data.
submit.to_csv('submission.csv', index=False)

# <p style="font-size: 24px; font-weight: bold;">Congratulations!</p>

<p style="font-size: 16px;">You're now ready to submit your work on Kaggle!</p>

<p style="font-size: 16px;">Enjoy your experience on Kaggle!</p>